In [1]:
import numpy as np
import pandas as pd
import re
import requests
from io import StringIO
import mysql.connector as mysql

from statistics import mode
from transliterate import translit
from workalendar.europe import Russia

from clickhouse_driver import Client
from sqlalchemy import create_engine, text
from clickhouse_sqlalchemy import make_session
from datetime import datetime, date, time
import ast  # Модуль ast (Abstract Syntax Trees) для преобразования строк в списки
from math import radians, sin, cos, sqrt, atan2

In [2]:
print('np.__version__', np.__version__)
print('pd.__version__', pd.__version__)
print('re.__version__', re.__version__)
print('requests.__version__', requests.__version__)
print('mysql.__version__', mysql.__version__)
from transliterate import __version__
print('transliterate.__version__', __version__)
from workalendar import __version__
print('workalendar.__version__', __version__)
from clickhouse_driver import __version__
print('clickhouse_driver.__version__', __version__)
from sqlalchemy import __version__
print('sqlalchemy.__version__', __version__)
from clickhouse_sqlalchemy import __version__
print('clickhouse_sqlalchemy.__version__', __version__)

np.__version__ 1.26.4
pd.__version__ 2.0.3
re.__version__ 2.2.1
requests.__version__ 2.31.0
mysql.__version__ 8.3.0
transliterate.__version__ 1.10.2
workalendar.__version__ 17.0.0
clickhouse_driver.__version__ 0.2.7
sqlalchemy.__version__ 2.0.28
clickhouse_sqlalchemy.__version__ 0.3.0


In [3]:
def get_from_synDB(query):
    conn = mysql.connect(
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
            )
    # Чтение данных из базы данных и запись их в DataFrame
    data = pd.read_sql_query(query, conn)

    # Закрытие соединения с базой данных
    conn.close()

    return data

def get_order_lines():

    order_lines = get_from_synDB("""
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
                            ;""")
    
    print('order_lines.shape', order_lines.shape)
    return order_lines

def get_orders():

    orders = get_from_synDB("""
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
                    ;""")
    
    print('orders.shape', orders.shape)
    return orders


def get_products():

    products = get_from_synDB("""
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
                          ;""")
    
    print('products.shape', products.shape)
    return products


def get_clients():

    clients = get_from_synDB("""
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
                        ;""")

    print('clients.shape', clients.shape)
    return clients


# Функция для вычисления разницы цен с конкурентами (использовать, если не будет решена проблема с нулевыми ценами)
def calculate_price_diff(row, price_competitor_column, column_syn_ru):
    if np.isnan(row[price_competitor_column]) or np.isnan(row[column_syn_ru]) or row[column_syn_ru] == 0:
        return 0
    else:
        result = (row[price_competitor_column] - row[column_syn_ru]) / row[column_syn_ru]
        if result > 1:
            return 1
        else:
            return round(result, 2)

def get_pricing():

    pricing = get_from_synDB("""
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
                        ;""")

    pricing.drop_duplicates(inplace=True)

    pricing['date'] = pd.to_datetime(pricing['date'])
    # Сортируем по дате
    pricing.sort_values(by='date', inplace=True)
    # Приводим к единому формату id
    pricing['id'] = pricing['id'].astype(float).astype(int)

    # Преобразуем столбец 'price_ozon' и 'price_wb' в тип данных float, с заменой некорректных значений на NaN
    pricing['price_ozon'] = pd.to_numeric(pricing['price_ozon'], errors='coerce')
    pricing['price_wb'] = pd.to_numeric(pricing['price_wb'], errors='coerce')

    # Считаем относительную разницу цен (отрицательное значение значит цену ниже, чем у , положительное - выше)
    pricing['price_ozon_diff'] = pricing.apply(lambda row: calculate_price_diff(row, 'price_ozon', 'price_syn_ru'), axis=1)
    pricing['price_wb_diff'] = pricing.apply(lambda row: calculate_price_diff(row, 'price_wb', 'price_syn_ru'), axis=1)

    # Создаем новый столбец, содержащий наибольшую разницу c 'price_ozon_diff' и 'price_wb_diff',
    # предполагая, что чем ниже цена на какой-то из этих площадок, тем более нечевствителен к скидкам покупающий на * клиент
    pricing['price_diff'] = abs(pricing[['price_ozon_diff', 'price_wb_diff']].min(axis=1))
    
    print('pricing.shape', pricing.shape)
    return pricing

def get_products_visits():

    products_visits = get_from_synDB("""
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
                             ;""")

    products_visits['date_time'] = pd.to_datetime(products_visits['date_time'])
    products_visits['ProductIdsWebsite'] = products_visits['ProductIdsWebsite'].astype('int32')

    return products_visits

def get_visits():

    engine = create_engine("
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
                           ", connect_args={'connect_timeout': 1800000})

    # Выгружаем только значимые столбцы с ограничение Bounce == 0 (если 1 - то отказ, их не учитываем)
    # Исключаем данные до 20-08-2022
    query = text("""
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
    """)
    session = make_session(engine)
    result = session.execute(query).fetchall()
    data = [row for row in result]
    visits = pd.DataFrame(data)

    print('visits.shape', visits.shape)
    return visits

def get_yandex_client_ids():
    yandex_client_ids = get_from_synDB("""
'''
ИСКЛЮЧЕНЫ ДАННЫЕ ПО NDA
'''
                                ;""")

    print('yandex_client_ids.shape', yandex_client_ids.shape)

    # Убираем дубликаты, возможные из-за отсутсвия столбца id
    yandex_client_ids.drop_duplicates(inplace=True)
    
    print('yandex_client_ids.shape', yandex_client_ids.shape)

    return yandex_client_ids

def get_nearest_pricing_time(df, pricing, date_time_column_name):
    price_list = pricing['date'].unique()
    # Создаем колонку с ближайшей датой назначения цены
    for i in df.index:
        try:
            df.loc[i, 'nearest_pricing_date'] = pd.to_datetime(price_list[price_list < df.loc[i, date_time_column_name]][-1])
        except:
            continue

    print('get_nearest_pricing_time.shape', df.shape)
    return df

def preprocess_products_for_merge(products):
    # Меняем тип данных для корректного мерджа
    products['ProductIdsWebsite'] = products['ProductIdsWebsite'].astype('int').astype('str')

    # Создаем столбец с кол-вом присвоенных категория для каждого товара
    def count_elements_in_list(lst):
        if lst is None:  # Добавляем проверку на значение None
            return 0
        else:
            return len(lst)
    products['number_of_categories'] =  products['ProductCategoriesIdsWebsite'].apply(count_elements_in_list)

    # Заменяем на основную категорию
    products['ProductCategoriesIdsWebsite'] = products['ProductCategoriesIdsWebsite'].str.split('|', expand=True)[0]

    print('products.shape', products.shape)
    return products    

def preprocess_yandex_client_ids(yandex_client_ids):
    yandex_client_ids.drop_duplicates(inplace=True)
    # Приводим в правильный формат даты
    yandex_client_ids['authorized_at'] = pd.to_datetime(yandex_client_ids['authorized_at'])
    # Корректно отображаем пустые значения
    yandex_client_ids.loc[:,['yandex_id', 'mindbox_id']] = yandex_client_ids.loc[:,['yandex_id', 'mindbox_id']].replace('', np.nan)
    # Вместо 'authorized_at' введем:
    # Первая авторизация
    yandex_client_ids['first_authorized_at'] = yandex_client_ids.groupby('mindbox_id')['authorized_at'].transform('min')
    # Последняя авторизация
    yandex_client_ids['last_authorized_at'] = yandex_client_ids.groupby('mindbox_id')['authorized_at'].transform('max')
    # Число проведенных на сайте дней
    yandex_client_ids['authorized_days'] = yandex_client_ids['authorized_at'].dt.date
    yandex_client_ids['authorized_days'] = yandex_client_ids.groupby('mindbox_id')['authorized_days'].transform('nunique')
    # Число авторизаций website_id
    yandex_client_ids['authorized_website_id_count'] = yandex_client_ids.groupby('mindbox_id')['website_id'].transform('count')
    yandex_client_ids.drop(columns=['website_id', 'authorized_at'], inplace=True)
    yandex_client_ids.drop_duplicates(inplace=True)
    # Сколько прошло дней с момента последней авторизации
    yandex_client_ids['days_since_last_authorized'] = (datetime.now() - yandex_client_ids['last_authorized_at']).dt.days

    print('yandex_client_ids.shape', yandex_client_ids.shape)

    return yandex_client_ids

def prepare_yandex_client_ids_to_merge(yandex_client_ids):

    # Готовим данные к мерджу
    yandex_client_ids_to_merge = yandex_client_ids.drop(columns=['yandex_id', 'first_authorized_at', 'last_authorized_at'])
    yandex_client_ids_to_merge.drop_duplicates(inplace=True)
    yandex_client_ids_to_merge['mindbox_id'] = pd.to_numeric(yandex_client_ids_to_merge['mindbox_id'])
    yandex_client_ids_to_merge.rename(columns={'mindbox_id': 'CustomerIdsMindboxId'}, inplace=True)

    print('yandex_client_ids_to_merge.shape', yandex_client_ids_to_merge.shape)

    return yandex_client_ids_to_merge

# Создаем словарь замены неуникальных id юзеров на один уникальный
def make_id_dic(id_series):
    id_dic = {}
    for row in id_series:
        if row[0] in id_dic.keys():
            for i in range(1, len(row)):
                id_dic[row[i]] = id_dic[row[0]]
        else:
            for i in range(1, len(row)):
                id_dic[row[i]] = row[0]
    # Делаем еще один проход, чтобы убрать лишние промежуточные ключи, созданные при первом проходе
    for key, value in id_dic.items():
        if value in id_dic.keys():
            id_dic[key] = id_dic[value]
    return id_dic

def make_yandex_client_ids_dic(yandex_client_ids):
    # Удаляем оставшиеся NAN
    yandex_client_ids = yandex_client_ids.dropna()

    # Дубликатов меньше в yandex_id, поэтому создадим справочник повторных присвоений mindbox_id и дальше уникальным клиентам присвоим уникальны mindbox_id 
    # (поскольку иногда возникает несколько для одного клиента)
    # В дальнейшем используя данный словарь, нужно будет избавиться от дубликатов и в других данных
    mindbox_id_series = yandex_client_ids[yandex_client_ids['yandex_id'].duplicated(keep=False)].groupby('yandex_id')['mindbox_id'].apply(list)

    # Сортируем одинаково, чтобы не возникало незаметных дубликатов
    mindbox_id_series = mindbox_id_series.apply(lambda x: sorted(x))

    # Удаляем дубликаты
    mindbox_id_series.drop_duplicates(keep='last', inplace=True)

    # Создаем словарь для замены неуникальных mindbox_id
    mindbox_id_dic = make_id_dic(mindbox_id_series)

    # Унифицируем mindbox_id
    yandex_client_ids.loc[:,'mindbox_id'] = yandex_client_ids['mindbox_id'].apply(lambda x: mindbox_id_dic[x] if x in mindbox_id_dic.keys() else x)
    yandex_client_ids.drop_duplicates(subset=['yandex_id', 'mindbox_id'], keep='last', inplace=True)
    yandex_client_ids.reset_index(inplace=True)

    # Создаем словарь для замены yandex_id на mindbox_id
    yandex_id_dic = {}
    for i in range(len(yandex_client_ids)):
        yandex_id_dic[yandex_client_ids.loc[i, 'yandex_id']] = yandex_client_ids.loc[i, 'mindbox_id']

    print('len(yandex_id_dic)', len(yandex_id_dic))

    return yandex_id_dic

    # Функция для удаления пустых значений из списка
def remove_empty_values(coupon_list):
    return len(list(filter(lambda x: x != '', coupon_list)))

# Функция для извлечения доменных имен с использованием регулярных выражений
def extract_domain(url):
    if pd.notna(url):
        match = re.search(r'[^.]+[.][^.]+$', url)
        return match.group() if match else None
    else:
        return None
    
def preprocess_visits(visits, yandex_id_dic):
    # Создаем столбец с id_mindbox
    visits.loc[:,'ClientID'] = visits['ClientID'].astype('str')
    visits.loc[:,'CustomerIdsMindboxId'] = visits['ClientID'].apply(lambda x: yandex_id_dic[x] if x in yandex_id_dic.keys() else np.nan)
    # Исключаем из данных пользователей, которых нет в базе mindbox')
    visits = visits[visits['CustomerIdsMindboxId'].notna()]
    # Приводим данные к нужному виду')
    visits.loc[:,'Date'] = pd.to_datetime(visits['Date'])
    # Восстановим NAN там, где они фактически присутствуют')
    visits.loc[:,'LastReferalSource'] = visits['LastReferalSource'].replace('', np.nan)
    visits.loc[:,'LastSearchEngineRoot'] = visits['LastSearchEngineRoot'].replace('', np.nan)
    visits.loc[:,'LastSocialNetwork'] = visits['LastSocialNetwork'].replace('', np.nan)
    visits.loc[:,'RegionCity'] = visits['RegionCity'].replace('', np.nan)
    # Создаем паттерн регулярного выражения для поиска текста между '(' и ','
    pattern = r'\((.*?)\,'
    # Функция, которая извлекает первое значение между '(' и ','
    def extract_value(text):
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        return None
    # Применяем функцию к столбцу visits['LastSourceEngine']
    visits.loc[:,'LastSourceEngine'] = visits['LastSourceEngine'].apply(lambda x: extract_value(x))
    # Считаем кол-во примененных купонов
    # От str переходим к list')
    visits.loc[:,'PurchaseCoupon'] = visits['PurchaseCoupon'].apply(ast.literal_eval)
    # Кол-во успешных применений купона')
    visits.loc[:,'PurchaseCoupon'] = visits['PurchaseCoupon'].apply(remove_empty_values)
    # Из PurchaseID извлекаем число заказов (в самих ID здесь смысла нет)')
    visits.loc[:,'PurchaseID'] = visits['PurchaseID'].apply(ast.literal_eval)
    visits.loc[:,'PurchaseID'] = visits['PurchaseID'].apply(lambda x: len(x))
    # Считаем кол-во WatchIDs (поскольку сами id ничему не релевантны)')
    visits.loc[:,'WatchIDs'] = visits['WatchIDs'].apply(lambda x: len(x))
    # Сокращаем значение до домена')
    visits.loc[:,'LastReferalSource'] = visits['LastReferalSource'].apply(extract_domain)
    # Рассчитываем частоту значений в столбце LastReferalSource')
    value_counts = visits['LastReferalSource'].value_counts(normalize=True)
    # Заменяем значения, которые встречаются реже, чем в 0.001 случаях, на Other')
    threshold = 0.001
    visits.loc[:,'LastReferalSource'] = visits['LastReferalSource'].apply(lambda x: x if value_counts.get(x, 0) >= threshold else 'Other')
    # Рассчитываем частоту значений в столбце LastSearchEngineRoot')
    value_counts = visits['LastSearchEngineRoot'].value_counts(normalize=True)
    visits.loc[:,'LastSearchEngineRoot'] = visits['LastSearchEngineRoot'].apply(lambda x: x if value_counts.get(x, 0) >= threshold else 'Other')
    # Рассчитываем частоту значений в столбце LastSocialNetwork')
    value_counts = visits['LastSocialNetwork'].value_counts(normalize=True)
    visits.loc[:,'LastSocialNetwork'] = visits['LastSocialNetwork'].apply(lambda x: x if value_counts.get(x, 0) >= threshold else 'Other')

    print('visits.shape', visits.shape)

    # Необходимо избавиться от дубликатов в данных. Поскольку метрика выдает интегральные показатели за сутки, ')
    # необходимо произвести слияние информации по визитам, совершенным в течение 1 суток
    visits = visits.groupby(['Date', 'CustomerIdsMindboxId']).agg({'DeviceCategory':'last', 'IsNewUser':'max', 'LastReferalSource':'last',
       'LastSearchEngineRoot':'last', 'LastSocialNetwork':'last', 'LastTrafficSource':'last',
       'PageViews':'sum', 'RegionCity':'last',
       'VisitDuration':'sum', 'VisitID':'count', 'WatchIDs':'sum', 'LastSourceEngine':'last', 
       'PurchaseCoupon':'sum', 'PurchaseID':'sum'}).reset_index()

    print('visits.shape', visits.shape)

    # Готовим к мерджу')
    visits = visits.rename(columns={'Date':'OrderDate',
                        'VisitID':'VisitIDNums', 'WatchIDs':'WatchIDsNums', 
                        'PurchaseCoupon':'PurchaseCouponApplied',
                        'PurchaseID':'PurchaseNums'})
    visits['CustomerIdsMindboxId'] = pd.to_numeric(visits['CustomerIdsMindboxId'])

    return visits

def get_sex_names():
    file_id = '1npMAUX-BiaXyVrHG_GogSEthSuQhtdeF'
    url = f'https://drive.google.com/uc?id={file_id}'
    response = requests.get(url)
    content = response.text
    file_like_object = StringIO(content)
    female_names = [line.lower().strip() for line in file_like_object.readlines()]

    file_id = '15VOP4TAW_IWZeSmkEvwpKRyS3ZXR5bKe'
    url = f'https://drive.google.com/uc?id={file_id}'
    response = requests.get(url)
    content = response.text
    file_like_object = StringIO(content)
    male_names = [line.lower().strip() for line in file_like_object.readlines()]

    return female_names, male_names

def contains_name(name, female_names, male_names):
    # Функция проверки имени на муж/жен
    if isinstance(name, float):
        return name
    words = name.lower().split()
    if any(word in female_names for word in words):
        return 'female'
    elif any(word in male_names for word in words):
        return 'male'
    else:
        return np.nan
    
def contains_surname(name):
    # Функция проверки фамилии на муж/жен
    if isinstance(name, float):
        return name
    words = name.lower().split()
    if any(word.endswith(('ев', 'ов', 'ин', 'ын')) for word in words):
        return 'male'
    elif any(word.endswith(("ева", "ова", "ина", 'ына', 'ая')) for word in words):
        return 'female'
    else:
        return np.nan
    
def contains_test(name):
    # Функция поиска тестовых значений
    if isinstance(name, float):
        return 1    
    if 'тест' in name:
        return np.nan
    elif 'проверка' in name:
        return np.nan
    else:
        return 1


def get_sex(df):

    # Определяем пол по окончаниям отчеств:
    df['CustomerSex2'] = df.CustomerMiddleName.apply(lambda x: 'male' if isinstance(x, str) 
                                                    and x.lower().endswith(('ич', 'ыч', 'лы'))
                                                    else 'female' if isinstance(x, str) 
                                                    and x.lower().endswith(("на", "зы", "ва"))
                                                    else np.nan)
    df['CustomerSex'] = df['CustomerSex'].fillna(df['CustomerSex2'])

    # Определяем пол по окончаниям фамилий:
    df['CustomerSex2'] = df.CustomerLastName.apply(lambda x: 'male' if isinstance(x, str) 
                                                and x.lower().endswith(('ев', 'ов', 'ин', 'ын'))
                                                else 'female' if isinstance(x, str) 
                                                and x.lower().endswith(("ева", "ова", "ина", 'ына', 'ая'))
                                                else np.nan)
    df['CustomerSex'] = df['CustomerSex'].fillna(df['CustomerSex2'])

    # Проведем транслитерацию имен в кириллицу с использование библиотеки transliterate
    df['CustomerFirstName_mod'] = df['CustomerFirstName'].fillna('')
    df['CustomerFirstName_mod'] = df['CustomerFirstName_mod'].apply(lambda x: translit(x.lower(), 'ru'))

    # Вычленим пол из имен
    female_names, male_names = get_sex_names()
    df['CustomerSex2'] = df['CustomerFirstName_mod'].apply(lambda x: contains_name(x, female_names, male_names))
    df['CustomerSex'] = df['CustomerSex'].fillna(df['CustomerSex2'])

    # Проверка окончаний на муж/жен
    df['CustomerSex2'] = df['CustomerFirstName_mod'].apply(lambda x: 'male' if isinstance(x, str) and ('ич' in x.lower() or 'ыч' in x.lower())
                                else 'female' if isinstance(x, str) and ("на" in x.lower() or "зы" in x.lower())
                                else 'female' if isinstance(x, str) and x.lower().endswith(("а", "я"))
                                else 'male' if isinstance(x, str) and x.lower().endswith(('й', 'р', 'н', 'в'))
                                else np.nan)
    df['CustomerSex'] = df['CustomerSex'].fillna(df['CustomerSex2'])

    # Вычленим пол из фамилий в поле имени
    df['CustomerSex2'] = df['CustomerFirstName_mod'].apply(lambda x: contains_surname(x))
    df['CustomerSex'] = df['CustomerSex'].fillna(df['CustomerSex2'])

    # Меняем на булевое: 1 - мужчина (небольшое число пропусков признаем женщинами, учитывая сильный перевес)
    df['IsMale'] = df['CustomerSex'] == 'male'

    print('get_sex.shape', df.shape)
    return df


def delete_test(df):
    # Удалим тестовые значения
    df['CustomerFirstName_mod'] = df['CustomerFirstName_mod'].apply(lambda x: contains_test(x))

    # Удалим строки, в которых столбец 'CustomerLastName' содержит слово, подобное "Тест"
    pattern = re.compile(r'.*Тест.*')
    df.loc[:, 'CustomerFirstName_mod'] = df['CustomerLastName'].fillna('')
    df = df[~df['CustomerFirstName_mod'].str.contains(pattern)]

    # Удалим строки, в которых столбец 'CustomerMiddleName' содержит слово, подобное "Тест"
    pattern = re.compile(r'.*Тест.*')
    df.loc[:, 'CustomerFirstName_mod'] = df['CustomerMiddleName'].fillna('')
    df = df[~df['CustomerFirstName_mod'].str.contains(pattern)]

    # Удалим строки, в которых столбец 'CustomerFirstName' содержит слово, подобное "Тест"
    pattern = re.compile(r'.*Тест.*')
    df.loc[:, 'CustomerFirstName_mod'] = df['CustomerFirstName'].fillna('')
    df = df[~df['CustomerFirstName_mod'].str.contains(pattern)]

    print('delete_test.shape', df.shape)
    return df

def extract_city(input_string):
    # Выделим города, очистив от лишних символов, названий регионов, улиц и т.д.
    if isinstance(input_string, str) and 'г ' in input_string:
        # Ищем 'г '
        index = input_string.find('г ')
        if index != -1:
            # Проверка на наличие пробела перед 'г '
            if index == 0 or (index > 0 and (input_string[index - 1] == ' ' or input_string[index - 1] == ',')):
                # Выделяем слова после 'г ' без запятой
                remaining_text = input_string[index + 2:]
                match = re.search(r'([\w-]+(?: [\w-]+)?)', remaining_text)
                if match:
                    return match.group(1).strip()
    return input_string

def extract_city_2(input_string):
    if isinstance(input_string, str) and 'г ' in input_string:
        # Найдем 'г '
        index = input_string.find('г ')
        if index != -1:
            # Проверим пробел перед 'г '
            if index == 0 or (index > 0 and (input_string[index - 1] == ' ' 
                                             or input_string[index - 1] == ',')):
                # Найдем значения после 'г '
                remaining_text = input_string[index + 2:]
                match = re.search(r'([\w-]+(?: [\w-]+)?)', remaining_text)
                if match:
                    return match.group(1).strip()
    return None

def extract_region(input_string):
    if (not isinstance(input_string, str)) or len(input_string) < 5:
        return None

    _obl = input_string.find('обл')
    _kray = input_string.find('край')
    _resp = input_string.find(' Респ,')
    _respublika = input_string.find(' республика')
    resp_ = re.search(r'Респ\s(\w+)', input_string)
    _ao = input_string.find(' АО')
    
    if 'Москва' in input_string:
        return 'Москва'
    elif 'Санкт-Петербург' in input_string:
        return 'Санкт-Петербург'
    elif _obl != -1:
        return input_string[:_obl].strip()
    elif _kray != -1:
        return input_string[:_kray].strip()
    elif _resp != -1:
        return input_string[:_resp].strip()
    elif _respublika != -1:
        return input_string[:_respublika].strip()
    elif resp_:
        return resp_.group(1)
    elif _ao != -1:
        return input_string[:_ao].strip()
    else:
        return None


def get_cities_names():
    file_id = '1ZGTMel-tik1z9SfkcNThc-FH5yawhm_4'
    url = f'https://drive.google.com/uc?id={file_id}'
    cities_ru_en = pd.read_csv(url, header=None)

    return cities_ru_en

def get_cities(df):
    cities_ru_en = get_cities_names()

    df.CustomerCustomFieldsCity = df.CustomerCustomFieldsCity.apply(extract_city)

    # RegionCity - город из яндекс метрики
    df = df.merge(cities_ru_en, how='left', left_on='RegionCity', right_on=1)
    df['CustomerCustomFieldsCity'] = df['CustomerCustomFieldsCity'].fillna(df[0])
    df.drop(columns=['RegionCity', 0, 1], inplace=True)

    # CustomerCustomFieldsCityOrder	Город доставки	object	265918	98.93	1	[Санкт-Петербург] - недостоверный 
    df['CustomerCustomFieldsCity'] = df['CustomerCustomFieldsCity'].fillna(df['CustomerCustomFieldsCityOrder'])

    # CustomerCustomFieldsCitySystem	Город (по номеру телефона)
    df = df.merge(cities_ru_en, how='left', left_on='CustomerCustomFieldsCitySystem', right_on=1)
    df['CustomerCustomFieldsCity'] = df['CustomerCustomFieldsCity'].fillna(df[0])
    df.drop(columns=['CustomerCustomFieldsCityOrder', 'CustomerCustomFieldsCitySystem', 0, 1], inplace=True)

    df['City'] = df['OrderCustomFieldsDeliveryAddress'].apply(extract_city_2)

    print('get_cities.shape', df.shape)
    return df

def get_bools(df):
    # Заменяем значения на булевые
    df['VK_bot'] = df.CustomerCustomFieldsUtmFirst.apply(lambda x: True if x == 'VK_bot' else False) # пришел с вконтакте
    df['Site'] = df.CustomerCustomFieldsUtmFirst.apply(lambda x: True if x == 'Site' else False) # пришел с сайта
    df['GreenWave'] = df.CustomerCustomFieldsUtmFirst.apply(lambda x: True if x == 'GreenW' else False) # пришел с Зеленой волны
    df['consultants'] = df.CustomerCustomFieldsUtmFirst.apply(lambda x: True if x == 'consultants' else False) # привлекли консультанты
    df['eco'] = df.CustomerCustomFieldsUtmFirst.apply(lambda x: True if x == 'ecoMobile' or x == 'sborka' else False) # привлечен эко-тематикой
    df['consultants_2'] = df.CustomerCustomFieldsUtmMedium.apply(lambda x: True if x == 'consultants' or x == 'consultanys' else False)
    df['consultants'] = df['consultants'] + df['consultants_2']
    df['consultants'] = df['consultants'].apply(lambda x: True if x >= 1 else False)
    df['eco_2'] = df.CustomerCustomFieldsUtmMedium.apply(lambda x: True if x in ['ecoMobile', 'sborka', 'Шоурум', 'Sborka', 'ecodom'] else False)
    df['eco'] = df['eco'] + df['eco_2']
    df['eco'] = df['eco'].apply(lambda x: True if x >= 1 else False)
    df = df.drop(columns=['CustomerCustomFieldsUtmFirst', 'CustomerCustomFieldsUtmMedium', 'consultants_2', 'eco_2'])

    # Заменим значения на булевые и переименуем:
    # 1, если было взаимодействие offline
    # 0 в других случаях
    df.CustomerCustomFieldsUtmSource = df.CustomerCustomFieldsUtmSource.apply(lambda x: True if x == 'offline'
                                                                            else False)
    df.rename(columns={'CustomerCustomFieldsUtmSource': 'offline'}, inplace=True)

    print('get_bools.shape', df.shape)
    return df


def process_regions(df):
    # extract region
    df['Region'] =  df['OrderCustomFieldsDeliveryAddress'].apply(extract_region)

    # Необходимо убрать города и прочий неадекват:
    df['Region'] = df['Region'].apply(lambda x: np.nan if str(x).startswith('г') else x)

    # Исправляем единичную ошибку с Еврейская А
    df['Region'] = df['Region'].apply(lambda x: 'Еврейская' if x == 'Еврейская А' else x)

    # Обнулим значения региона, если они прошли с ошибкой (частотность значения меньше 15)
    value_counts = df['Region'].value_counts()
    values_to_replace = value_counts[value_counts < 15].index
    df['Region'] = df['Region'].apply(lambda x: np.nan if x in values_to_replace else x)

    # Заполним пустые города (поставив условие, что регион неизвестен, потому что необходимо, 
    # чтобы не возникло противоречия между городом и регионом 
    # (такое возможно, поскольку CustomerCustomFieldsCity неточный предсказатель)

    df['City'] = df['City'].fillna(df[df['Region'].isna()]['CustomerCustomFieldsCity'])

    # Оставшиеся пропуски считаем сельской местностью. Присвоим значение 'Сельские районы'
    df['City'] = df['City'].fillna('Сельские районы')

    print('process_regions.shape', df.shape)
    return df


def get_cities_db():
    # Считываем нужные колонки из базы данных городов России
    file_id = '1ETmId0_xHc71dYCL78zdZI69Q94VtDgN'
    url = f'https://drive.google.com/uc?id={file_id}'
    cities = pd.read_csv(url,  usecols=['federal_district', 'region_type', 'region', 'city', 'capital_marker', 'timezone', 'geo_lat', 'geo_lon', 'population'])

    # Меняем на признак, является ли национальным образованием
    cities['region_type'] = cities['region_type'].apply(lambda x: 0 if x == 'обл' or x == 'край' or x == 'г'
                            else 1)

    # Заполняем города, где не указаны
    for i in cities[cities.city.isna()].index:
        if cities.region[i] == 'Москва':
            cities.loc[i, 'city'] = 'Москва'
        elif cities.region[i] == 'Санкт-Петербург':
            cities.loc[i, 'city'] ='Санкт-Петербург'
        elif cities.region[i] == 'Севастополь':
            cities.loc[i, 'city'] ='Севастополь'

    # Удаляем пустые значения
    cities = cities.dropna(subset=['city'])

    # Подрезаем названия регионов, оставляем только первое слово
    cities.region = cities.region.apply(lambda x: x.split()[0])

    # Поскольку в качестве ключа будем использовать город, избавляемся от дубликатов, оставляя более населенные города
    cities = cities.sort_values(by='population')
    cities = cities.drop_duplicates(subset='city', keep='last')

    return cities


def timezone_to_regions(df):
    # Остаток регионов вытянем из таймзоны
    TimeZone_to_Region = df.groupby('CustomerIanaTimeZone')['Region'].agg(mode).reset_index()
    df = df.merge(TimeZone_to_Region, how='left', on='CustomerIanaTimeZone')
    df.Region_x = df.Region_x.fillna(df.Region_y)
    df = df.drop(columns='Region_y')
    df = df.rename(columns={'Region_x': 'Region'})

    print('timezone_to_regions.shape', df.shape)
    return df


def process_capital_marker(df):
    # Переделаем capital_marker следующим образом:
    # поднимем города вверх на 1 ранг, 0 присвоим сельской местности
    # присвоим городам федерального значения ранг 4 - Москва, Санкт-Петербург, 3 - Севастополь (на уровне регионального центра)
    df.capital_marker = df.capital_marker.apply(lambda x: x+1)

    for i in df[(df.City == 'Москва')].index:
        df.loc[i, 'capital_marker'] = 4
    for i in df[(df.City == 'Санкт-Петербург')].index:
        df.loc[i, 'capital_marker'] = 4
    for i in df[(df.City == 'Севастополь')].index:
        df.loc[i, 'capital_marker'] = 3

    df.capital_marker = df.capital_marker.fillna(0).astype('int')

    print('process_capital_marker.shape', df.shape)
    return df

# Функция для исправления региона с Москвы на Мос обл для значений не "Москва"
def fix_fedcity_obl(row, city, region):
    if (row[region] ==  'Москва') & (row[city] != 'Москва'):
        return 'Московская'
    elif (row[region] ==  'Санкт-Петербург') & (row[city] != 'Санкт-Петербург'):
        return 'Ленинградская'
    elif (row[region] ==  'Севастополь') & (row[city] != 'Севастополь'):
        return 'Крым'
    else:
        return row[region]

def add_geo_data(df):
    # После всех дополнений осталось совсем мало незаполненных значений. Заполним их модой
    region_na_index = df[df.Region.isna()].index
        
    region_mode = mode(df['Region'].dropna())
    df.Region = df.Region.fillna(region_mode)

    # Также в пропущенных регионах заполним модой город
    city_mode = mode(df['City'].dropna())
    for i in region_na_index:
        df.loc[i, 'City'] = city_mode

    # Заполняем города-регионы, где не указаны
    for i in df.City[df.City.isna()].index:
        if df.Region[i] == 'Москва':
            df.loc[i, 'City'] = 'Москва'
        elif df.Region[i] == 'Санкт-Петербург':
            df.loc[i, 'City'] ='Санкт-Петербург'
        elif df.Region[i] == 'Севастополь':
            df.loc[i, 'City'] ='Севастополь'

    # Делаем поправку на Москву
    df['Region'] = df.apply(lambda row: fix_fedcity_obl(row, 'City', 'Region'), axis=1)

    print('add_geo_data.shape', df.shape)
    return df


def process_city_popul(df):
    # Заполним отсутствующие данные 'City_popul' средним по региону, если это город, и 0, если это не город
    city_uniques = df[(df.City_popul.isna()) & (df.City != 'Сельские районы')].City.unique()
    region_means = df.groupby(['Region', 'City'])['City_popul'].first().reset_index().groupby('Region')['City_popul'].mean()

    for city in city_uniques:
        city_index = df[df.City == city].index
        for i in city_index:
            df.loc[i, 'City_popul'] = int(region_means[df.loc[i, 'Region']])

    df.City_popul = df.City_popul.fillna(0)

    # Заменяем абсолютные значения на ранги
    df['City_popul'] = df['City_popul'].apply(lambda x: x if x == 0
                           else 1 if x > 0 and x <= 50_000
                           else 2 if x > 50_000 and x <= 100_000
                           else 3 if x > 100_000 and x <= 250_000
                           else 4 if x > 250_000 and x <= 500_000
                           else 5 if x > 500_000 and x <= 750_000
                           else 6 if x > 750_000 and x <= 1_000_000
                           else 7 if x > 1_000_000 and x <= 3_000_000
                           else 8 if x > 3_000_000 and x <= 7_000_000
                           else 9)
    
    print('process_city_popul.shape', df.shape)    
    return df

# Функция замены числа городов в регионе для федеральных городов
def replace_num_cities(df):
    mask = df['Region'] == 'Москва'
    df.loc[mask, 'Num_of_cities'] = (df.loc[df['Region'] == 'Московская', 'Num_of_cities'].values + 1).mean()
    mask = df['Region'] == 'Санкт-Петербург'
    df.loc[mask, 'Num_of_cities'] = (df.loc[df['Region'] == 'Ленинградская', 'Num_of_cities'].values + 1).mean()
    mask = df['Region'] == 'Севастополь'
    df.loc[mask, 'Num_of_cities'] = (df.loc[df['Region'] == 'Крым', 'Num_of_cities'].values + 1).mean()

    print('replace_num_cities.shape', df.shape)     
    return df

def haversine_distance(lat1, lon1, lat2, lon2):
    # Конвертируем координаты из градусов в радианы
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Радиус Земли в километрах
    R = 6371.0

    # Разница в координатах
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Формула гаверсинусового расстояния
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Расстояние между точками
    distance = R * c
    return distance

def add_federal_district_data(df, cities):
    # Дообогащаем данные по федеральным округам доставки и таймзонам
    regions_federal_district = cities.groupby('region').agg({
        'federal_district': 'first',  # принадлежность федеральному округу
        'region_type': 'first',  # тип региона
        'city': 'count',    # кол-во городов в регионе (как показатель урбанизированности)
        'population': 'sum'  # число жителей в регионе
    }).reset_index()
    regions_federal_district = regions_federal_district.rename(columns={'city':'Num_of_cities', 'population':'Region_popul', 'region':'Region'})
    
    # Исправляем число городов в федеральных городах
    regions_federal_district = regions_federal_district.pipe(replace_num_cities)

    # Считываем нужные колонки из базы данных ВРП по регионам России за 2023г (https://rosstat.gov.ru/folder/13397)
    file_id = '182OjhZqF80AGpKcUlcANo-iZJbLAw1jR'
    url = f'https://drive.google.com/uc?id={file_id}'
    vrp = pd.read_excel(url)

    # Переводим ВРП на душу населения в тыс.руб.
    vrp['vrp_per_person'] = round(vrp['vrp_per_person']/1000)

    # Дополняем данные по регионам ВРП на душу населения
    regions_federal_district = regions_federal_district.merge(vrp, how='inner', on='Region')

    df = df.merge(regions_federal_district, how='left', on='Region')
    df = df.drop(columns=['OrderCustomFieldsDeliveryAddress'])

    # Заменяем абсолютные значения Region_popul на ранги
    df['Region_popul'] = df['Region_popul'].apply(lambda x: 1 if x <= 100_000
                           else 2 if x > 100_000 and x <= 250_000
                           else 3 if x > 250_000 and x <= 500_000
                           else 4 if x > 500_000 and x <= 750_000
                           else 5 if x > 750_000 and x <= 1_000_000
                           else 6 if x > 1_000_000 and x <= 3_000_000
                           else 7 if x > 3_000_000 and x <= 7_000_000
                           else 8)
    
    # Заменяем пропуски в координатах средними по региону
    df['geo_lat'] = df['geo_lat'].fillna(df.groupby('Region')['geo_lat'].transform('mean'))
    df['geo_lon'] = df['geo_lon'].fillna(df.groupby('Region')['geo_lon'].transform('mean'))

    # Заданные координаты
    lat2, lon2 = df[df['City'] == 'Москва'].loc[0, 'geo_lat'], df[df['City'] == 'Москва'].loc[0, 'geo_lon']

    # Создание функции для расчета расстояния от заданной точки до каждой строки DataFrame
    def calculate_distance(row):
        lat1, lon1 = row['geo_lat'], row['geo_lon']
        return haversine_distance(lat1, lon1, lat2, lon2)

    # Применение функции к каждой строке и создание нового столбца 'distance'
    df['distance'] = df.apply(calculate_distance, axis=1)

    print('add_federal_district_data.shape', df.shape)    
    return df


def add_delivery_type(df):
    df.Delivery = df.Delivery.apply(
        lambda x: 1 if x in ['58', '48', '78', '87', '2', '46', 'Привезет курьер']
        else 0
    )
    df['payment_type'] = pd.to_numeric(df['payment_type'], errors='coerce')

    # Создаем новый признак "Покупка в кредит"
    df['Credit'] = df['payment_type'] == 11

    # Создаем новый признак "Оплачен при получении" (остальные - оплачены в момент заказа на сайте)
    df['Paid_on_delivery'] = df['payment_type'] == 1

    print('add_delivery_type.shape', df.shape)
    return df


def fill_na_with_modified_value(price_dic, row):
    if pd.isna(row['ProductName']):
        return price_dic[row['OrderLineBasePricePerItem']]
    return row['ProductName']


def recover_product_name(df):
    # Можно попробовать восстаносить пропуски ProductName, используя совпадения по OrderLineBasePricePerItem
    # Заменим nan наиболее частотным товаром по данной цене
    na_product_name_prices = df[df.ProductName.isna()].OrderLineBasePricePerItem.unique()
    price_dic = {}
    for price in na_product_name_prices:
        try:
            price_dic[price] = df[df.OrderLineBasePricePerItem == price].OrderLineProductName.value_counts().head(1).index[0]
        except:
            price_dic[price] = 'unknown'

    df['ProductName'] = df.apply(lambda row: fill_na_with_modified_value(price_dic, row), axis=1)

    print('recover_product_name.shape', df.shape)
    return df


def recover_timezone(df):
    # Восстановим таймзону CustomerIanaTimeZone из информации о регионах
    Region_to_TimeZone = df.groupby('Region')['CustomerIanaTimeZone'].agg(mode).reset_index()
    df = df.merge(Region_to_TimeZone, how='left', on='Region')
    df.CustomerIanaTimeZone_x = df.CustomerIanaTimeZone_x.fillna(df.CustomerIanaTimeZone_y)
    df = df.drop(columns='CustomerIanaTimeZone_y')
    df = df.rename(columns={'CustomerIanaTimeZone_x': 'CustomerIanaTimeZone'})

    print('recover_timezone.shape', df.shape)
    return df


def fill_scores(df):
    df['CustomerCustomFieldsScorBaby'] = df['CustomerCustomFieldsScorBaby'].fillna(False)
    df['CustomerCustomFieldsScorBeauty'] = df['CustomerCustomFieldsScorBeauty'].fillna(False)
    df['CustomerCustomFieldsScorStirka'] = df['CustomerCustomFieldsScorStirka'].fillna(False)
    df['CustomerCustomFieldsScorUborka'] = df['CustomerCustomFieldsScorUborka'].fillna(False)
    # Приводим к нужному типу данных
    df['CustomerCustomFieldsScorBaby'] = df['CustomerCustomFieldsScorBaby'].astype('bool')
    df['CustomerCustomFieldsScorBeauty'] = df['CustomerCustomFieldsScorBeauty'].astype('bool')
    df['CustomerCustomFieldsScorStirka'] = df['CustomerCustomFieldsScorStirka'].astype('bool')
    df['CustomerCustomFieldsScorUborka'] = df['CustomerCustomFieldsScorUborka'].astype('bool')

    print('fill_scores.shape', df.shape)
    return df


def fill_nans(df):
    df['payment_type'] = df['payment_type'].fillna(mode(df['payment_type']))
    df['ProductCategoriesIdsWebsite'] = df['ProductCategoriesIdsWebsite'].fillna(0)
    df['price_syn_ru'] = df['price_syn_ru'].fillna(df['OrderLineBasePricePerItem'])
    df['price_diff'] = df['price_diff'].fillna(0)
    df['authorized_days'] = df['authorized_days'].fillna(round(df['authorized_days'].mean()))
    df['authorized_website_id_count'] = df['authorized_website_id_count'].fillna(round(df['authorized_website_id_count'].mean()))
    df['days_since_last_authorized'] = df['days_since_last_authorized'].fillna(round(df['days_since_last_authorized'].mean()))
    df['DeviceCategory'] = df['DeviceCategory'].fillna(mode(df['DeviceCategory'].dropna())).astype('int')
    df['LastReferalSource'] = df['LastReferalSource'].fillna('Other')
    df['LastSearchEngineRoot'] = df['LastSearchEngineRoot'].fillna('Other')
    df['LastSocialNetwork'] = df['LastSocialNetwork'].fillna('Other')
    df['LastTrafficSource'] = df['LastTrafficSource'].fillna('Other')
    df['PageViews'] = df['PageViews'].fillna(round(df['PageViews'].mean()))
    df['VisitDuration'] = df['VisitDuration'].fillna(round(df['VisitDuration'].mean()))
    df['VisitIDNums'] = df['VisitIDNums'].fillna(round(df['VisitIDNums'].mean()))
    df['WatchIDsNums'] = df['WatchIDsNums'].fillna(round(df['WatchIDsNums'].mean()))
    df['LastSourceEngine'] = df['LastSourceEngine'].fillna(-10).astype('int')
    df['PurchaseCouponApplied'] = df['PurchaseCouponApplied'].fillna(0)

    df = df.loc[~((df['OrderLineStatusIdsExternalId'] == 'Cancelled') | (df['OrderLineStatusIdsExternalId'] == 'Mindbox-Tech-Cancellation'))]

    print('fill_nans.shape', df.shape)
    return df


def add_calendar_features(df):
    # Устанавливаем правильный формат
    df.OrderFirstActionDateTimeUtc = pd.to_datetime(df.OrderFirstActionDateTimeUtc)

    # Создадим дополнительные временные признаки
    df['OrderDate'] = pd.to_datetime(df['OrderFirstActionDateTimeUtc'].dt.date)
    df['OrderYear'] = df['OrderFirstActionDateTimeUtc'].dt.year
    df['OrderMonth'] = df['OrderFirstActionDateTimeUtc'].dt.month
    df['OrderDay'] = df['OrderFirstActionDateTimeUtc'].dt.day
    df['OrderHour'] = df['OrderFirstActionDateTimeUtc'].dt.hour
    df['OrderMinute'] = df['OrderFirstActionDateTimeUtc'].dt.minute
    df['OrderDayOfWeek'] = df['OrderFirstActionDateTimeUtc'].dt.weekday
    df['OrderWeekNumber'] = df['OrderFirstActionDateTimeUtc'].dt.isocalendar().week

    # Рабочий или праздничный/выходной
    cal = Russia() # Использование календаря для России
    df['IsWorkingDay'] = df['OrderFirstActionDateTimeUtc'].apply(cal.is_working_day)

    print('add_calendar_features.shape', df.shape)
    return df


def get_clear_price(df):
    # Имеет смысл ввести новое поле для расчета общей стоимости заказа без вычетов по скидкам и кэшбекам, чтобы убрать это искажение. 
    # Кроме того, кажется разумным исключить из стоимости заказа цену доставки, поскольку эта цена не является предметом выбора покупателя, 
    # а зависит исключительно от порога стоимости заказа и региона.

    # Сгруппируем данные по заказам
    df_total_price = df.groupby('OrderIdsMindboxId', as_index=False).agg({
        'OrderTotalPrice': 'first', 
        'OrderDeliveryCost': 'first', 
        'OrderLinePriceOfLine': 'sum'})
    df_total_price = df_total_price.rename(columns={'OrderLinePriceOfLine' : 'OrderTotalPrice_cleared'})
    df_total_price['OrderTotalPrice_cleared'] = df_total_price['OrderTotalPrice_cleared'].round(2)

    # Добавим новый столбец 'OrderTotalPrice_cleared' к df
    # Дальнейшее исследование будем проводить на этом очищенном признаке

    df = df.merge(df_total_price[['OrderIdsMindboxId', 'OrderTotalPrice_cleared']], 
                                on='OrderIdsMindboxId', how='left')
    
    # Сгруппируем данные по заказам
    df_total_price = df.groupby('OrderIdsMindboxId', as_index=False)['OrderTotalPrice_cleared'].first()

    # Вычисляем среднее и стандартное отклонение для 'OrderTotalPrice'
    mean_price = df_total_price['OrderTotalPrice_cleared'].mean()
    std_deviation = df_total_price['OrderTotalPrice_cleared'].std()

    # Задаем пороговое значение Z-Score
    threshold = 2.5

    # Рассчитываем Z-Score для каждой точки данных
    df_total_price['Z_Score'] = (df_total_price['OrderTotalPrice_cleared'] - mean_price) / std_deviation

    # Выделяем точки данных, у которых Z-Score выше порога
    outliers = df_total_price[df_total_price['Z_Score'] > threshold]

    # Пороговое значение OrderTotalPrice_cleared
    threshold_total_price = int(outliers['OrderTotalPrice_cleared'].min())

    # Хвосту справа имеет смысл присвоить значение, по которому происходит отсечение.
    df_total_price.loc[df_total_price['OrderTotalPrice_cleared'] > threshold_total_price, 'OrderTotalPrice_cleared'] = threshold_total_price
    df_total_price.rename(columns={'OrderTotalPrice_cleared': 'OrderTotalPrice_cleared_Z'}, inplace=True)

    # Добавим новый столбец 'OrderTotalPrice_cleared_Z' к df, где максимальные значения OrderTotalPrice_cleared заменены пороговым

    df = df.merge(df_total_price[['OrderIdsMindboxId', 'OrderTotalPrice_cleared_Z']], 
                                on='OrderIdsMindboxId', how='left')
    
    print('get_clear_price.shape', df.shape)    
    return df


def drop_tests(df):
    # Присутствуют некорректные значения, видимо, относящиеся к тестированию сервиса. Их нужно удалить из списка заказов.
    orders_to_drop = list(df[(df.ProductName == 'unknown') 
                                    | (df.ProductName == 'тестовый')].OrderIdsMindboxId.index)
    df = df.drop(index=orders_to_drop)

    print('drop_tests.shape', df.shape) 
    return df


def process_quantity(df):
    # Вычисляем среднее и стандартное отклонение для 'OrderLineQuantity'
    df['OrderLineQuantity'] = df['OrderLineQuantity'].astype('int')
    mean_price = df['OrderLineQuantity'].mean()
    std_deviation = df['OrderLineQuantity'].std()

    # Задаем пороговое значение Z-Score
    threshold = 2

    # Рассчитываем Z-Score для каждой точки данных
    df['Z_Score'] = (df['OrderLineQuantity'] - mean_price) / std_deviation

    # Выделяем точки данных, у которых Z-Score выше порога
    outliers = df[df['Z_Score'] > threshold]

    df.drop(columns='Z_Score', inplace=True)

    # Зададим максимальное пороговое значение
    threshold_line_quantity = int(outliers.OrderLineQuantity.min())
    print(f"Threshold value OrderLineQuantity: {threshold_line_quantity}")

    # Хвосту справа имеет присвоим значение, по которому происходит отсечение.
    df['OrderLineQuantity_Z'] = df['OrderLineQuantity']
    df.loc[df['OrderLineQuantity'] > threshold_line_quantity, 'OrderLineQuantity_Z'] = threshold_line_quantity

    print('process_quantity.shape', df.shape)
    return df


def process_price(df):
    # Базовая цена никогда не должна быть меньше продажной цены. Для сравнения добавим новый признак 'PriceOfLinePerItem'
    df['PriceOfLinePerItem'] = df['OrderLinePriceOfLine'] / df['OrderLineQuantity']

    # Рассчитаем полный размер скидки
    df['DiscountAmount'] = round(1 - (df['PriceOfLinePerItem'] / (df['OrderLineBasePricePerItem'] + 0.01)), 4) * 100

    # Основные статистики DiscountAmount в разрезе по товарам
    product_describe_df = df.groupby('ProductIdsWebsite')['DiscountAmount'].describe().applymap(lambda x: float(f'{x:.2f}')).reset_index()
    # Добавляем границы межквартильного размаха
    df = df.merge(product_describe_df[['ProductIdsWebsite', '25%', '75%']], how='left')
    # Исправляем тип данных
    df['25%'] = df['25%'].astype('float')
    df['75%'] = df['75%'].astype('float')
    # Считаем, выпадает ли размер скидки из межквартильного размаха
    df['25%'] = (df['DiscountAmount'] <= df['25%'])* -1  # 
    df['75%'] = df['DiscountAmount'] >= df['75%']
    df['Сustomer_discount_rank'] = df['25%'] + df['75%']
    # Считаем среднюю "склонность к скидкам" для каждого клиента
    customer_discount_rank_df = df.groupby('CustomerIdsMindboxId')['Сustomer_discount_rank'].mean().reset_index()
    # Удаляем лишние столбцы из дф
    df.drop(columns=['25%', '75%', 'Сustomer_discount_rank'], inplace=True)
    # Добавляем расчитанный столбец 'Сustomer_discount_rank' к основному дф
    df = df.merge(customer_discount_rank_df, how='left')
    
    print('process_price.shape', df.shape)
    return df


def check_gift(df):
    # Создаем новый признак, означающий наличие подарка
    df['Gift_Inside'] = df['OrderLinePriceOfLine'] == 0
    # Общее число, полученных клиентом
    df['cust_gift_sum'] = df.groupby('CustomerIdsMindboxId')['Gift_Inside'].transform('sum')
    # Соотношение подарков к платным товарам у клиента
    df['cust_gift_mean'] = df.groupby('CustomerIdsMindboxId')['Gift_Inside'].transform('mean')
    # Как часто товар предлагается клиентам в качестве подарка
    df['prod_as_gift_mean'] = df.groupby('ProductIdsWebsite')['Gift_Inside'].transform('mean')
    # Число подарков в заказе
    df['Gift_Inside'] = df.groupby('OrderIdsMindboxId')['Gift_Inside'].transform('sum')

    print('check_gift.shape', df.shape)
    return df


def check_sale(df):
    # Новый признак, показывающий, был ли внутри товар с 99% сидкой: 'Sale99_Inside'
    # Создаем новый признак - товар продан со скидкой > 98%
    df['Sale99_Inside'] = ((df['DiscountAmount'] > 98) & (df['DiscountAmount'] < 100))

    # Создаем группировку с кол-вом акционных товаров в каждом заказе
    sale99 = df.groupby('OrderIdsMindboxId')['Sale99_Inside'].sum().reset_index()

    # Булевый признак переименовываем и меняем на кол-во продаж товара с 99% скидкой 
    df = df.rename(columns={'Sale99_Inside': 'product_sale99'})
    df['product_sale99'] = df.groupby('ProductIdsWebsite')['product_sale99'].transform('sum')

    # Добавляем столбец с кол-вом товаров со скидкой 99% в заказе
    df = df.merge(sale99, on='OrderIdsMindboxId', how='left')

    # Добавляем столбец со средним числом товаров со скидкой 99% в заказах клиента
    Customer_Sale99_Inside = df.groupby(['CustomerIdsMindboxId', 'OrderIdsMindboxId'])['Sale99_Inside'].first().groupby('CustomerIdsMindboxId').mean().reset_index()
    Customer_Sale99_Inside = Customer_Sale99_Inside.rename(columns={'Sale99_Inside':'cust_sale99_mean'})
    df = df.merge(Customer_Sale99_Inside, on='CustomerIdsMindboxId', how='left')

    # Добавляем столбец со средним числом товаров со скидкой 99% в заказах клиента
    Customer_Sale99_Inside = df.groupby(['CustomerIdsMindboxId', 'OrderIdsMindboxId'])['Sale99_Inside'].first().groupby('CustomerIdsMindboxId').sum().reset_index()
    Customer_Sale99_Inside = Customer_Sale99_Inside.rename(columns={'Sale99_Inside':'cust_sale99_sum'})
    df = df.merge(Customer_Sale99_Inside, on='CustomerIdsMindboxId', how='left')

    print('check_sale.shape', df.shape)
    return df

def replace_value_cities(pop_value, x):
    if x in pop_value:
        return x
    else:
        return 'Города <0.1%'
    

def replace_value_regions(pop_value, x):
    if x in pop_value:
        return x
    else:
        return 'Регионы <0.5%'
    
def replace_value_zones(pop_value, x):
    if x in pop_value:
        return x
    else:
        return 'Other <0.5%'



def drop_non_active_goods(df):
    # Исключаем данные до 20-08-2022 
    df_paid = df[df['OrderFirstActionDateTimeUtc'] > '20-08-2022']

    # Исключаем неактивные товары
    df_paid = df_paid[df_paid['ProductIsAvailable'] == 1]
    df_paid.drop(columns='ProductIsAvailable', inplace=True)

    # Исключаем товары с 100% и 99% скидкой
    df_paid = df_paid[df_paid['OrderLinePriceOfLine'] != 0]
    df_paid.drop(columns='OrderLinePriceOfLine', inplace=True)
    df_paid = df_paid[df_paid['DiscountAmount'] < 98]

    # Добавляем дополнительный целевой признак - для классификации
    df_paid['Сustomer_discount_rank_class'] = df_paid['Сustomer_discount_rank'].apply(lambda x: 0 if x < -0.3
                                                                            else 2 if x > 0.3
                                                                            else 1)

    # Меняем порядок столбцов для удобства
    new_column_order = df_paid.columns.to_list()
    new_column_order.remove('Сustomer_discount_rank')
    new_column_order.remove('Сustomer_discount_rank_class')
    new_column_order = ['Сustomer_discount_rank', 'Сustomer_discount_rank_class'] + new_column_order

    df_paid = df_paid[new_column_order]

    print('df_paid.shape', df_paid.shape)
    return df_paid


def get_product_rank(df):
    # Ранее мы исключили из рассмотрения Cancelled заказы (отмененные). поскольку абсолютное большинство из них были признаны тестовыми.
    # Также, вероятно, из данных следует исключить вновь созданные, но еще не обработанные заказы, поскольку часть из них также может быть невалидна.
    # (Например, им еще просто не успели назначить статус Cancelled)
    # После этого столбец стоит удалить.

    df = df[df['OrderLineStatusIdsExternalId'] != 'Create']
    df = df.drop(columns='OrderLineStatusIdsExternalId')

    # Из OrderLineNumber можно извлечь следующую информацию:
    # 1) Общее число уникальных товаров в заказе
    # 2) Порядок добавления товара в корзину (что скорее всего стоит интерпретировать как значимость товара для этого заказа / клиента)

    # Очевидно, нам важен порядок добавления только для первых товаров, далее значимость этого признака снижается.
    # Разобьем данные на перцентили с шагом в 20%, чтобы получить разумные рамки для укрупнения рангов:
    percentiles = [i for i in range(0, 101, 20)]

    # Вычисляем перцентили
    percentile_values = df['OrderLineNumber'].quantile([p/100 for p in percentiles])

    # Создаем агрегированный DataFrame
    agg_df = pd.DataFrame({'Percentile': percentiles, 'OrderLineNumber': percentile_values})
    agg_df['OrderLineNumber'] = agg_df['OrderLineNumber'].apply(lambda x: f'{x:.0f}')

    # Также необходимо учесть, что товар мог добавляться не во все заказы, 
    # т.е. для правильного расчета ранга нужно учесть и те случаи, когда товар в корзину добавлен не был,
    # и присвоить ему в этих случаях "последнее" место.

    # Создадим новый признак 'ProductRank' на основе распределения рангов на перцентили:
    df['ProductRank'] = df['OrderLineNumber'].apply(lambda x: 1 if x == 1
                                                                else 2 if x >= 2 and x < 4
                                                                else 3 if x >= 4 and x < 7
                                                                else 4 if x >= 7 and x < 11
                                                                else 5 if x >= 11
                                                                else np.nan)
    
    print('get_product_rank.shape', df.shape)
    return df


def process_order_lines(df):
    # Посчитаем число строк (продуктов внутри) по заказам
    lines_inside = df.groupby('OrderIdsMindboxId')['OrderLineNumber'].max().reset_index()
    lines_inside = lines_inside.rename(columns={'OrderLineNumber': 'LineNumbersInside'})

    # Добавим новый признак, означающий число товарных позиций в заказе - 'LineNumbersInside'
    df = df.merge(lines_inside, on='OrderIdsMindboxId', how='left')
    # Вычисляем среднее и стандартное отклонение для 'LineNumbersInside'
    mean_number = lines_inside['LineNumbersInside'].mean()
    std_deviation = lines_inside['LineNumbersInside'].std()
    # Задаем пороговое значение Z-Score
    threshold = 2

    # Рассчитываем Z-Score для каждой точки данных
    lines_inside['Z_Score'] = (lines_inside['LineNumbersInside'] - mean_number) / std_deviation
    # Выделяем точки данных, у которых Z-Score выше порога
    outliers = lines_inside[lines_inside['Z_Score'] > threshold]
    # Присвоим вместо экстремальных значений пороговые
    threshold_line_quantity = int(outliers['LineNumbersInside'].min())

    # Хвосту справа имеет смысл присвоить значение, по которому происходит отсечение.
    lines_inside.loc[lines_inside['LineNumbersInside'] > threshold_line_quantity, 'LineNumbersInside'] = threshold_line_quantity  
    # Добавим новый признак, означающий число товаров в заказе - 'LineNumbersInside'
    lines_inside = lines_inside.rename(columns={'LineNumbersInside': 'LineNumbersInside_Z'})
    lines_inside.drop(columns='Z_Score', inplace=True)
    df = df.merge(lines_inside, on='OrderIdsMindboxId', how='left')

    print('process_order_lines.shape', df.shape)
    return df


def post_processing(df):
    # Часть городов имеют очень низкую частотность, заменим их на единое значение
    df_total_price = df.groupby('OrderIdsMindboxId', as_index=False)['City'].first()
    # Список городов, составляющих > 0.1%
    value_counts = df_total_price['City'].value_counts(normalize=True)
    pop_value = value_counts[value_counts > 0.001].index.tolist()
    # Применяем функцию к столбцу 'City'
    df['City'] = df['City'].apply(lambda x: replace_value_cities(pop_value, x))

    # Часть регионов имеют очень низкую частотность, заменим их на единое значение
    df_total_price = df.groupby('OrderIdsMindboxId', as_index=False)['Region'].first()
    # Список регионов, составляющих > 0.5%
    value_counts = df_total_price['Region'].value_counts(normalize=True)
    pop_value = value_counts[value_counts > 0.005].index.tolist()
    # Применяем функцию к столбцу 'Region'
    df['Region'] = df['Region'].apply(lambda x: replace_value_regions(pop_value, x))

    # Часть зон имеют очень низкую частотность, заменим их на единое значение
    df_total_price = df.groupby('OrderIdsMindboxId', as_index=False)['CustomerIanaTimeZone'].first()
    # Список зон, составляющих > 0.5%
    value_counts = df_total_price['CustomerIanaTimeZone'].value_counts(normalize=True)
    pop_value = value_counts[value_counts > 0.005].index.tolist()
    # Применяем функцию к столбцу 'CustomerIanaTimeZone'
    df['CustomerIanaTimeZone'] = df['CustomerIanaTimeZone'].apply(lambda x: replace_value_zones(pop_value, x))

    # Дни рождения заполнены менее, чем у 4% активных клиентов
    df_birthdays = df.groupby('CustomerIdsMindboxId')['CustomerBirthDate'].first()
    # Есть значения, превышающие 100 лет или не достигшие совершеннолетия - скорее всего данные недостоверны, удалим эти значения
    # Посчитаем возраст клиентов
    df['CustomerBirthDate'] = pd.to_datetime(df['CustomerBirthDate'])
    current_date = pd.to_datetime('now') # текущая дата
    # Возраст приближенно в годах
    df['Age'] = round((current_date - df['CustomerBirthDate']) / pd.Timedelta(days=365.25))
    # Удалим недостоверные значения
    df['Age'] = df['Age'].apply(lambda x: x if x > 16 and x < 100 else np.nan)

    df = df.drop(columns=['CustomerCustomFieldsEcoMobile'])

    # 'CustomerCustomFieldsGWtickets'
    data=df.groupby('CustomerIdsMindboxId')['CustomerCustomFieldsGWtickets'].first().value_counts().reset_index()
    # Вычисляем первый (25-й процентиль) и третий (75-й процентиль) квартили
    Q1 = data['CustomerCustomFieldsGWtickets'].quantile(0.25)
    Q3 = data['CustomerCustomFieldsGWtickets'].quantile(0.75)
    # Вычисляем межквартильный диапазон (IQR)
    IQR = Q3 - Q1
    # Задаем пороговые значения для определения выбросов
    lower_threshold = Q1 - 2 * IQR
    upper_threshold = Q3 + 2 * IQR
    # Выделяем точки данных, находящиеся за пределами порогов
    outliers = data[(data['CustomerCustomFieldsGWtickets'] < lower_threshold) | (data['CustomerCustomFieldsGWtickets'] > upper_threshold)]
    # Данные без выбросов
    data=data[(data['CustomerCustomFieldsGWtickets'] > lower_threshold) & (data['CustomerCustomFieldsGWtickets'] < upper_threshold)]
    outliers_list = list(outliers['CustomerCustomFieldsGWtickets'].unique())
    # Уберем экстремальные значения, заменив средней
    GWtickets_mean = int(data['CustomerCustomFieldsGWtickets'].mean())
    df['CustomerCustomFieldsGWtickets'] = df['CustomerCustomFieldsGWtickets'].apply(lambda x: GWtickets_mean if x in outliers_list
                                                 else x)
    
    df.drop(columns=['OrderLineNumber',
                        'OrderLineBasePricePerItem', # корреляция с price_syn_ru 0.943247
                        'OrderTotalPrice',
                        'payment_type',
                        'ProductName',
                        'CustomerFirstName',
                        'CustomerMiddleName', 
                        'CustomerLastName',
                        'CustomerBirthDate',
                        'PriceOfLinePerItem',
                        ],
                        inplace=True)

    print('post_processing.shape', df.shape)
    return df


def get_pricing_stat(pricing):

    pricing_stat = pricing[['id', 'price_syn_ru', 'date']].sort_values(by=['id', 'date'])
    # Группируем данные по 'id' и вычисляем разницу между соседними датами
    pricing_stat['date_diff'] = pricing_stat.groupby('id')['date'].diff()
    # Используем метод shift, чтобы сдвинуть значения на одну позицию вперед
    pricing_stat['date_diff'] = pricing_stat.groupby('id')['date_diff'].shift(-1)
    # Последние значения (NaT) заполняем нулями
    pricing_stat['date_diff'] = pricing_stat['date_diff'].fillna(pd.Timedelta(0))

    # Считаем время экспозиции каждой цены в днях
    pricing_stat = pricing_stat.groupby(['id', 'price_syn_ru'], as_index=False).agg(
        price_exp_pd = ('date_diff', 'sum')
        )
    pricing_stat['price_exp_pd'] = pricing_stat['price_exp_pd'].dt.total_seconds() / 60 / 60 / 24

    return pricing_stat

def add_product_stats(products_visits, df, pricing_stat):

    visits_count = products_visits.groupby(['ProductIdsWebsite', 'price_syn_ru'], as_index=False).agg(visits_count= ('date_time', 'count'))
    print('visits_count.shape', visits_count.shape)
    # Считаем число заказов по каждой цене
    orders_count = df[['OrderIdsMindboxId', 'OrderFirstActionDateTimeUtc', 'ProductIdsWebsite', 'price_syn_ru']][df['OrderFirstActionDateTimeUtc'] >= products_visits['date_time'].min()]
    orders_count = orders_count.groupby(['ProductIdsWebsite', 'price_syn_ru'], as_index=False).agg(orders_count= ('OrderIdsMindboxId', 'count'))
    print('orders_count.shape', orders_count.shape)

    pricing_stat = pricing_stat.merge(visits_count, left_on=['id', 'price_syn_ru'], right_on=['ProductIdsWebsite', 'price_syn_ru'], how='left')
    print('pricing_stat.merge(visits_count.shape', pricing_stat.shape)
    pricing_stat = pricing_stat.merge(orders_count, left_on=['id', 'price_syn_ru'], right_on=['ProductIdsWebsite', 'price_syn_ru'], how='left')
    print('pricing_stat.merge(orders_count.shape', pricing_stat.shape)
    pricing_stat.drop(columns=['ProductIdsWebsite_x', 'ProductIdsWebsite_y'], inplace=True)
    pricing_stat_notna = pricing_stat.fillna(0)

    # Посчитаем корреляции числа заказов товаров и цены, числа визитов, времени экспозиции на сайте.
    # Если корреляция не может быть корректно посчитана, считаем, что ее нет (равна 0)
    price_corr_id = pricing_stat_notna.groupby('id').apply(lambda x: x['orders_count'].corr(x['price_syn_ru'])).fillna(0)
    visits_corr_id = pricing_stat_notna.groupby('id').apply(lambda x: x['orders_count'].corr(x['visits_count'])).fillna(0)
    exp_pd_corr_id = pricing_stat_notna.groupby('id').apply(lambda x: x['orders_count'].corr(x['price_exp_pd'])).fillna(0)

    # Средние продажи в день (берем в знаменатель дни экспозиции товара)
    orders_count_id = pricing_stat_notna.groupby('id')['orders_count'].sum() / pricing_stat_notna.groupby('id')['price_exp_pd'].sum()

    # Вычисляем взвешенное среднее цены с учетом периодов действия внутри каждой группы 'id'
    avg_price_id = pricing_stat_notna.groupby('id').apply(lambda x: (x['price_syn_ru'] * x['price_exp_pd']).sum() / x['price_exp_pd'].sum())

    # Вычисляем стандартное отклонение цены 'price_syn_ru' по группам 'id'
    std_price_id = pricing_stat_notna.groupby('id')['price_syn_ru'].std()

    # Готовим к присоединению к единой таблице
    order_corr_id = pd.concat([avg_price_id, std_price_id, price_corr_id, visits_corr_id, exp_pd_corr_id, orders_count_id], axis=1).reset_index()
    order_corr_id.columns = ['ProductIdsWebsite','avg_price', 'std_price', 'price_corr', 'visits_corr', 'exp_pd_corr', 'orders_per_day']
    # Присваиваем в неопределенные корреляции средние по столбцам
    order_corr_id = order_corr_id.fillna(order_corr_id.mean())
    print('order_corr_id.shape', order_corr_id.shape)
    print('df.shape', df.shape)

    # Присоединяем новые признаки
    df = df.merge(order_corr_id, on='ProductIdsWebsite', how='left')
    print('df.merge(order_corr_id.shape', df.shape)

    return df

def fill_nans_and_add_features(df):

    # Общее фактическое число заказов клиента
    df['total_purchase_nums'] = df.groupby(['CustomerIdsMindboxId'])['OrderIdsMindboxId'].transform('nunique')

    # Первая/последняя покупка клиента
    df['cust_first_order'] = df.groupby('CustomerIdsMindboxId')['OrderFirstActionDateTimeUtc'].transform('first')
    df['cust_last_order'] = df.groupby('CustomerIdsMindboxId')['OrderFirstActionDateTimeUtc'].transform('last')

    # Среднее время между покупками клиента в днях
    df['cust_days_betw_orders'] = (df['cust_last_order'] - df['cust_first_order']).dt.total_seconds() / df['total_purchase_nums'] / 60 / 60 / 24

    # Число дней с первой/последней покупки
    df['cust_first_order'] = (pd.to_datetime('today') - df['cust_first_order']).dt.total_seconds() / 60 / 60 / 24
    df['cust_last_order'] = (pd.to_datetime('today') - df['cust_last_order']).dt.total_seconds() / 60 / 60 / 24

    # Средняя стоимость заказа клиента (средний чек)
    Customer_OrderTotalPrice_cleared = df.groupby(['CustomerIdsMindboxId', 'OrderIdsMindboxId'])['OrderTotalPrice_cleared'].first().groupby('CustomerIdsMindboxId').mean().reset_index()
    Customer_OrderTotalPrice_cleared = Customer_OrderTotalPrice_cleared.rename(columns={'OrderTotalPrice_cleared':'cust_total_price_mean'})
    df = df.merge(Customer_OrderTotalPrice_cleared, on='CustomerIdsMindboxId', how='left')

    # Общая стоимость заказов клиента
    Customer_OrderTotalPrice_cleared = df.groupby(['CustomerIdsMindboxId', 'OrderIdsMindboxId'])['OrderTotalPrice_cleared'].first().groupby('CustomerIdsMindboxId').sum().reset_index()
    Customer_OrderTotalPrice_cleared = Customer_OrderTotalPrice_cleared.rename(columns={'OrderTotalPrice_cleared':'cust_total_price_sum'})
    df = df.merge(Customer_OrderTotalPrice_cleared, on='CustomerIdsMindboxId', how='left')

    # Кол-во предыдущих заказов клиента
    CustomerIdsMindboxId_grouped = df.groupby(['CustomerIdsMindboxId', 'OrderIdsMindboxId'], as_index=False)['OrderFirstActionDateTimeUtc'].first()
    CustomerIdsMindboxId_cumcount = CustomerIdsMindboxId_grouped.groupby('CustomerIdsMindboxId').cumcount()
    CustomerIdsMindboxId_grouped = pd.concat([CustomerIdsMindboxId_grouped, CustomerIdsMindboxId_cumcount], axis=1)
    CustomerIdsMindboxId_grouped.rename(columns={0:'cust_orders_cumcount'}, inplace=True)
    df = df.merge(CustomerIdsMindboxId_grouped, on=['CustomerIdsMindboxId', 'OrderIdsMindboxId', 'OrderFirstActionDateTimeUtc'], how='left')

    # Общее число заказов товара
    df['product_total_order_num'] = df.groupby('ProductIdsWebsite')['OrderIdsMindboxId'].transform('count')

    # Общее кол-во проданного товара
    df['product_total_quantity'] = df.groupby('ProductIdsWebsite')['OrderLineQuantity'].transform('sum')

    # Средняя цена продажи товара
    df['product_avg_order_price'] = round(df.groupby('ProductIdsWebsite')['price_syn_ru'].transform('mean'), 2)

    # Среднее отклонение продажной цены товара
    df['product_std_order_price'] = round(df.groupby('ProductIdsWebsite')['price_syn_ru'].transform('std'), 2)

    # Время, в течение которого товар продается (в днях)
    df['product_selling_exp'] = (df.groupby('ProductIdsWebsite')['OrderFirstActionDateTimeUtc'].transform('last') \
                                        - df.groupby('ProductIdsWebsite')['OrderFirstActionDateTimeUtc'].transform('first')).dt.days

    # Заполняем NAN
    df['IsNewUser'] = df['IsNewUser'].astype('bool').fillna(df['cust_orders_cumcount'] < 1)
    df['PurchaseNums'] = df['PurchaseNums'].fillna(df['IsNewUser'] == 0).astype('int')
    df['avg_price'] = df['avg_price'].fillna(df['product_avg_order_price'])
    df['std_price'] = df['std_price'].fillna(df['product_std_order_price'])
    df['price_corr'] = df['price_corr'].fillna(0)
    df['visits_corr'] = df['visits_corr'].fillna(0)
    df['exp_pd_corr'] = df['exp_pd_corr'].fillna(0)
    df['orders_per_day'] = df['orders_per_day'].fillna(df['product_total_order_num'] / df['product_selling_exp'])
    df['Age'] = df['Age'].fillna(-1)

    print('fill_nans_and_add_features', df.shape)

    df.dropna(inplace=True)
    print('fill_nans_and_add_features', df.shape)

    return df


In [4]:
# load dfs
order_lines = get_order_lines()
orders = get_orders()
products = get_products()
clients = get_clients()
pricing = get_pricing()
products_visits = get_products_visits()
visits = get_visits()
yandex_client_ids = get_yandex_client_ids()

# Соединяем order_lines и orders
# Убирание updated_at в order_lines привело к возникновению дубликатов, вычистим их
df = order_lines[['ids.mindboxId', 
                'customer.ids.mindboxId',
                'firstAction.dateTimeUtc',
                'deliveryCost',
                'lines.product.ids.website',
                'lines.quantity',
                'lines.basePricePerItem',
                'lines.priceOfLine',
                'lines.status.ids.externalId',
                'lines.number']].drop_duplicates().merge(
                    orders[['ids.mindboxId', 
                            'totalPrice',
                            'customFields.deliveryAddress',
                            'customFields.deliveryType',
                            'payment_type']], how='inner')

print('df.shape', df.shape)

products = preprocess_products_for_merge(products)

df = df.merge(products, 
                    how='inner', 
                    left_on='lines.product.ids.website', 
                    right_on='ProductIdsWebsite')
df.drop(columns='ProductIdsWebsite', inplace=True)

print('df.shape', df.shape)

# Добавим данные по clients, исключив клиентов, которые не делали покупок, и тех, кого не получилось идентифицировать по id
df = df.merge(clients[['CustomerIdsMindboxId',
            'CustomerSex',
            'CustomerFirstName',
            'CustomerMiddleName',
            'CustomerLastName',
            'CustomerIanaTimeZone',
            'CustomerBirthDate',
            'CustomerIsEmailInvalid',
            'CustomerIsMobilePhoneInvalid',
            'CustomerBalanceLoyaltyProgramAccountTotal',
            'CustomerCustomFieldsCity',
            'CustomerCustomFieldsCityOrder',
            'CustomerCustomFieldsCitySystem',
            'CustomerCustomFieldsEcoMobile',
            'CustomerCustomFieldsGWtickets',
            'CustomerCustomFieldsLoyaltyCommunication',
            'CustomerCustomFieldsLoyaltyProduct',
            'CustomerCustomFieldsScorBaby',
            'CustomerCustomFieldsScorBeauty',
            'CustomerCustomFieldsScorStirka',
            'CustomerCustomFieldsScorUborka',
            'CustomerCustomFieldsUtmFirst',
            'CustomerCustomFieldsUtmMedium',
            'CustomerCustomFieldsUtmSource']], 
            left_on='customer.ids.mindboxId', 
            right_on='CustomerIdsMindboxId', 
            how='inner')
df = df.drop(columns='CustomerIdsMindboxId')

print('df.shape', df.shape)

# Меняем тип данных для дат
df['firstAction.dateTimeUtc'] = pd.to_datetime(df['firstAction.dateTimeUtc'])
# Приводим к единому формату id
df['lines.product.ids.website'] = df['lines.product.ids.website'].astype(float).astype(int)
# process pricing time
df = get_nearest_pricing_time(df, pricing, date_time_column_name='firstAction.dateTimeUtc')
# Добавляем данные по актуальным ценам во время заказа
df = df.merge(pricing[['id', 'date', 'price_syn_ru', 'price_diff']].drop_duplicates(),
        left_on=['lines.product.ids.website', 'nearest_pricing_date'],
        right_on=['id', 'date'],
        how='left')
df.drop(columns=['id', 'date'], inplace=True)

print('df.shape', df.shape)

# Переименуем колонки для удобства
df = df.rename(columns={'ids.mindboxId':'OrderIdsMindboxId',
                'customer.ids.mindboxId':'CustomerIdsMindboxId',
                'firstAction.dateTimeUtc':'OrderFirstActionDateTimeUtc', 
                'deliveryCost':'OrderDeliveryCost', 
                'lines.product.ids.website':'ProductIdsWebsite',
                'lines.quantity':'OrderLineQuantity', 
                'lines.basePricePerItem':'OrderLineBasePricePerItem', 
                'lines.priceOfLine':'OrderLinePriceOfLine',
                'lines.status.ids.externalId':'OrderLineStatusIdsExternalId', 
                'lines.number':'OrderLineNumber',
                'totalPrice':'OrderTotalPrice',
                'customFields.deliveryAddress':'OrderCustomFieldsDeliveryAddress',
                'customFields.deliveryType':'Delivery'})

# keep active orders only
df = add_calendar_features(df)

# Отсортируем данные по дате
df = df.sort_values(by=['OrderFirstActionDateTimeUtc', 'OrderIdsMindboxId', 'OrderLineNumber'])

# Обрабатываем yandex_client_ids
yandex_client_ids = preprocess_yandex_client_ids(yandex_client_ids)

# Добавляем данные из yandex_client_ids к df
yandex_client_ids_to_merge = prepare_yandex_client_ids_to_merge(yandex_client_ids)
df = df.merge(yandex_client_ids_to_merge, on='CustomerIdsMindboxId', how='left')

print('df.shape', df.shape)

# Создаем словарь замены yandex_id на mindbox_id
yandex_id_dic = make_yandex_client_ids_dic(yandex_client_ids)

# Добавляем данные из visits к df
visits = preprocess_visits(visits, yandex_id_dic)
df = df.merge(visits, on=['CustomerIdsMindboxId', 'OrderDate'], how='left')

print('df.shape', df.shape)

order_lines.shape (1012924, 19)
orders.shape (53457, 14)
products.shape (602, 4)
clients.shape (230150, 24)
pricing.shape (821595, 9)


visits.shape (2589765, 16)


yandex_client_ids.shape (160479, 4)
yandex_client_ids.shape (159535, 4)
df.shape (513591, 14)
products.shape (602, 5)
df.shape (509920, 18)
df.shape (457359, 41)
get_nearest_pricing_time.shape (457359, 42)
df.shape (457359, 44)
add_calendar_features.shape (457359, 53)
yandex_client_ids.shape (70814, 7)
yandex_client_ids_to_merge.shape (46544, 4)
df.shape (457359, 56)


len(yandex_id_dic) 41558
visits.shape (220599, 17)
visits.shape (141468, 16)
df.shape (457359, 70)


In [5]:
# extract customers' sex
df = get_sex(df)

# delete test rows
df = delete_test(df)

# Удалим ненужные столбцы
df = df.drop(columns=['CustomerSex', 'CustomerSex2', 'CustomerFirstName_mod'])

# convert birthdate to datetime
df.CustomerBirthDate = pd.to_datetime(df.CustomerBirthDate)

# Заполняем пустоты TODO separate function
df['CustomerIsEmailInvalid'] = df['CustomerIsEmailInvalid'].fillna(1) # пустые - невалидны
df['CustomerIsEmailInvalid'] = df['CustomerIsEmailInvalid'].astype('bool')
df['CustomerIsMobilePhoneInvalid'] = df['CustomerIsMobilePhoneInvalid'].fillna(1) # пустые - невалидны
df['CustomerIsMobilePhoneInvalid'] = df['CustomerIsMobilePhoneInvalid'].astype('bool')
df.CustomerBalanceLoyaltyProgramAccountTotal = df.CustomerBalanceLoyaltyProgramAccountTotal.fillna(0) # отстуствие баланса меняем на 0

# extract cities
df = get_cities(df)

# Заполним отсутсвующие значения 0 TODO separate function
df['CustomerCustomFieldsEcoMobile'] = df['CustomerCustomFieldsEcoMobile'].fillna(0)
df['CustomerCustomFieldsGWtickets'] = df['CustomerCustomFieldsGWtickets'].fillna(0)
df['CustomerCustomFieldsLoyaltyCommunication'] = df['CustomerCustomFieldsLoyaltyCommunication'].fillna(0)
df['CustomerCustomFieldsLoyaltyProduct'] = df['CustomerCustomFieldsLoyaltyProduct'].fillna(0)
# Приводим к нужному типу
df['CustomerCustomFieldsLoyaltyCommunication'] = df['CustomerCustomFieldsLoyaltyCommunication'].astype('int')
df['CustomerCustomFieldsLoyaltyProduct'] = df['CustomerCustomFieldsLoyaltyProduct'].astype('int')

# get boolean valeus
df = get_bools(df)

# Если не было доставки - стоимость 0
df.OrderDeliveryCost = df.OrderDeliveryCost.fillna(0) 

# process regions
df = process_regions(df)

# merge with cities db
cities = get_cities_db()

df = df.merge(cities[['region', 'city']], how='left', left_on='City', right_on='city')
df.Region = df.Region.fillna(df.region)
df = df.drop(columns=['region', 'city', 'CustomerCustomFieldsCity'])

# get regions from timezone
df = timezone_to_regions(df)

# add geo data
df = add_geo_data(df)

# Дообогащаем данные для городов
df = df.merge(cities[['city', 'capital_marker', 'population', 'geo_lat', 'geo_lon']], how='left', left_on='City', right_on='city')
df = df.drop(columns=['city'])
df = df.rename(columns={'population':'City_popul'})

# process capital markers
df = process_capital_marker(df)

# process city population
df = process_city_popul(df)

# add geo_lat, geo_lon, vrp, region_popul and other data based on federal districts
df = add_federal_district_data(df, cities)

# add delviery type data
df = add_delivery_type(df)

# recover product name
df = recover_product_name(df)

# recover timezone
df = recover_timezone(df)

# fill empty scores
df = fill_scores(df)

# fill NANs
df = fill_nans(df)

# get price cleared
df = get_clear_price(df)

# drop test rows
df = drop_tests(df)

df = process_quantity(df)

# process the price data
df = process_price(df)

get_sex.shape (457359, 73)
delete_test.shape (453123, 73)
get_cities.shape (453123, 68)
get_bools.shape (453123, 71)
process_regions.shape (453123, 72)
timezone_to_regions.shape (453123, 71)
add_geo_data.shape (453123, 71)
process_capital_marker.shape (453123, 75)
process_city_popul.shape (453123, 75)
replace_num_cities.shape (85, 5)
add_federal_district_data.shape (453123, 80)
add_delivery_type.shape (453123, 82)
recover_product_name.shape (453123, 82)
recover_timezone.shape (453123, 82)
fill_scores.shape (453123, 82)
fill_nans.shape (423199, 82)
get_clear_price.shape (423199, 84)
drop_tests.shape (423199, 84)
process_price.shape (423199, 87)


In [6]:
# add gift info
df = check_gift(df)

# add sale99 info
df = check_sale(df)

# get product rank
df = get_product_rank(df)

# get lines number 
df = process_order_lines(df)

# miscellaneous processing
df = post_processing(df)

# add product features
# Считаем число визитов по каждой цене
products_visits = get_nearest_pricing_time(products_visits, pricing, date_time_column_name='date_time')
print('products_visits.shape', products_visits.shape)


products_visits = products_visits.merge(pricing[['id', 'date', 'price_syn_ru']].drop_duplicates(),
         left_on=['ProductIdsWebsite', 'nearest_pricing_date'],
         right_on=['id', 'date'],
         how='left')
products_visits.drop(columns=['id', 'date'], inplace=True)
print('products_visits.shape', products_visits.shape)

# Считаем время экспозиции цен
pricing_stat = get_pricing_stat(pricing)
print('pricing_stat.shape', pricing_stat.shape)

df = add_product_stats(products_visits, df, pricing_stat)


df = fill_nans_and_add_features(df)

# drop inactive goods
df = drop_non_active_goods(df)

check_gift.shape (423199, 91)
check_sale.shape (423199, 95)
get_product_rank.shape (423199, 95)
process_order_lines.shape (423199, 97)
post_processing.shape (423199, 87)
get_nearest_pricing_time.shape (1420926, 3)
products_visits.shape (1420926, 3)
products_visits.shape (1420926, 4)
pricing_stat.shape (17798, 3)
visits_count.shape (12172, 3)
orders_count.shape (11489, 3)
pricing_stat.merge(visits_count.shape (17798, 5)
pricing_stat.merge(orders_count.shape (17798, 7)


order_corr_id.shape (561, 7)
df.shape (423199, 87)
df.merge(order_corr_id.shape (423199, 93)
fill_nans_and_add_features (423199, 105)
fill_nans_and_add_features (421814, 105)
df_paid.shape (372238, 104)
